# Distributed CBO -- Benchmark and Comparison

In [ ]:
%load_ext autoreload
%autoreload 2

import math
import time

import matplotlib.pyplot as plt
import numpy as np

from cbx.dynamics import CBO, DistributedCBO
from cbx.objectives import Rastrigin, Ackley, Michalewicz
import cbx.utils.termination as term

# CBO Configuration

In [ ]:
conf = {
    'alpha': 40.0,
    'dt': 0.1,
    'sigma': 1.,
    'lamda': 1.0,
    'd': 20,
    'term_criteria': [term.max_it_term(100)],
    'track_args': {'names': ['update_norm', 'energy', 'x', 'consensus', 'drift']},

    'noise': 'anisotropic',
    'f_dim': '3D'
}

# Define Benchmarking Functions

In [ ]:
def benchmark_distributed_cbo(
    f: callable,
    num_runs: int,
    max_splitting_factor: int,
    max_particles: int,
    synchronization_interval: int,
    synchronization_method: str,
    num_steps: int,
    use_async_communication: bool = False,
    axs=None,
):  
    energy_means = []
    execution_times = []
    for splitting_factor in range(1, max_splitting_factor + 1):
        num_particles = math.ceil(max_particles / splitting_factor)
        energies = []
        for _ in range(num_runs):
            dyn = DistributedCBO(
                f=f,
                num_agent_batches=splitting_factor,
                synchronization_interval=synchronization_interval,
                synchronization_method=synchronization_method,
                use_async_communication=use_async_communication,
                N=num_particles,
                verbose=False,
                max_it=10000,
                **conf
            )

            # Run optimization and measure time
            tick = time.time()
            energies.append(
                f(dyn.optimize(num_steps=num_steps, sched='default'))
            )
            tock = time.time()
            execution_times.append(tock - tick)

        mean_energy = np.mean(energies)
        print(f"Splitting factor: {splitting_factor}, mean energy: {mean_energy}. Execution time: {np.mean(execution_times)}")
        energy_means.append(np.mean(energies))  

    if axs is not None:
        async_label = 'Async' if use_async_communication else 'Sync'
        axs.plot(list(range(1, max_splitting_factor + 1)), energy_means, label=f'Distributed CBO {async_label}')


In [ ]:
def benchmark_standard_cbo(
    f: callable,
    num_runs: int,
    num_particles: int,
    axs=None,
):
    energies = []
    execution_times = []
    for _ in range(num_runs):
        dyn = CBO(
            f,
            N=num_particles,
            verbosity=0,
            max_it=10000,
            batch_args=None,
            M=1,
            **conf
        )

        # Run optimization and measure time
        tick = time.time()
        energies.append(
            f(dyn.optimize())
        )
        tock = time.time()
        execution_times.append(tock - tick)
    print(f"Non distributed mean energy: {np.mean(energies)}. Execution time: {np.mean(execution_times)}")

    # Plot mean of undistributed energies as a horizontal line
    if axs is not None:
        axs.axhline(y=np.mean(energies), color='r', linestyle='--', label='Standard CBO')


# Benchmark

In [ ]:
# Benchmarking params
NUM_RUNS = 100
MAX_SPLITTING_FACTOR = 10
MAX_PARTICLES = 100
SYNCHRONIZATION_INTERVAL = 10
SYNCHRONIZATION_METHOD = 'weighted_mean'

In [ ]:
def run_experiment(
    f: callable,
    d: int,
    n: int
):
    fig, axs = plt.subplots(1, 1)

    conf['d'] = d
    # Baseline (standard CBO)
    benchmark_standard_cbo(
        f=f,
        num_runs=NUM_RUNS,
        num_particles=n,
        axs=axs
    )

    # Synchronous
    benchmark_distributed_cbo(
        f=f,
        use_async_communication=False,
        num_runs=NUM_RUNS,
        max_splitting_factor=MAX_SPLITTING_FACTOR,
        max_particles=n,
        synchronization_interval=SYNCHRONIZATION_INTERVAL,
        synchronization_method=SYNCHRONIZATION_METHOD,
        num_steps=1000000,   # Arbitrarily high number since we depend on the termination criteria
        axs=axs
    )

    # Asynchronous
    benchmark_distributed_cbo(
        f=f,
        use_async_communication=True,
        num_runs=NUM_RUNS,
        max_splitting_factor=MAX_SPLITTING_FACTOR,
        max_particles=n,
        synchronization_interval=SYNCHRONIZATION_INTERVAL,
        synchronization_method=SYNCHRONIZATION_METHOD,
        num_steps=100,
        axs=axs
    )

    function_label = f.__class__.__name__ + f" dim={d}"
    axs.set_xlabel(f"Splitting factor")
    axs.set_ylabel(f"Mean energy")
    axs.set_title(f"{function_label}\nsync interval: {SYNCHRONIZATION_INTERVAL}, sync method: {SYNCHRONIZATION_METHOD}, particles: {MAX_PARTICLES}")
    axs.legend()
    
    fig.show()

## Ackley (d=5, N=100)

In [ ]:
run_experiment(Ackley(), d=5, n=100)

## Ackley (d=10, N=100)

In [ ]:
run_experiment(Ackley(), d=10, n=100)

## Ackley (d=20, N=100)

In [ ]:
run_experiment(Ackley(), d=20, n=100)